In [15]:
import pandas as pd
import convenient_methods as c_m
import convenient_lists as c_l
import linkedIn_scraper
import emoji



In [ ]:
##  Panda Methods ##



Make Folders, and then Scrape Data

Run if you want to check out the scraping process, etc.


In [ ]:
## Test Scraper ##
## Please only use characters that are acceptable to file names as an input. ##
## Should just use a number in the search quantity as well ##
c_m.create_folders()

search_input = input("Put in what you are looking for?")
search_quantity = int(input("At least, how many posts do you want to scrape?"))

scraping = True
while scraping:
    driver = linkedIn_scraper.scraper_log_in()
    linkedIn_scraper.search_and_scrape(search_input,search_quantity,driver)
    linkedIn_scraper.scraper_log_out(driver)
    scraping = False
search_input_list = [search_input]
c_m.json_organizer(search_input_list,search_input)
cleaned_posts_list = c_m.CSV_duplicate_finder(search_input_list,f'{search_input}_cleaned')
c_m.emoji_organizer(cleaned_posts_list,search_input)

In [ ]:
## Scraper used for project ##

c_m.create_folders()

scraping = True
while scraping:
    driver = linkedIn_scraper.scraper_log_in()
    for search in c_l.search_list:
        linkedIn_scraper.search_and_scrape(search,100,driver)
        print(f'{search} complete!')
    scraping = False
linkedIn_scraper.scraper_log_out(driver)

Converting JSONs to CSVs, cause i like them more.

In [ ]:
c_m.json_organizer(c_l.resume_json,'resume tips.csv')
c_m.json_organizer(c_l.interview_tips_json,'interview tips.csv')
c_m.json_organizer(c_l.data_analysis_tips_json,'data analysis tips.csv')
c_m.json_organizer(c_l.job_tips_json,'job tips.csv')

In [ ]:
## If wanting to run the duplicate finder one individual file, you can use this method ##
c_m.CSV_duplicate_finder(['resume tips'],'cleaned_resume_tips')
c_m.CSV_duplicate_finder(['interview tips'],'cleaned_interview_tips')
c_m.CSV_duplicate_finder(['data analysis tips'],'cleaned_data_analysis_tips')
c_m.CSV_duplicate_finder(['job tips'],'cleaned_job_tips')


In [3]:
## We want to pull the emojis from all NON DUPLICATED posts ##

post_list = c_m.CSV_duplicate_finder(c_l.csv_list,'All_Posts.csv')
c_m.emoji_organizer(post_list,"ALL_EMOJI_LIST")


All_Posts.csv has 1506 duplicates of 2436 posts
ALL_EMOJI_LIST.csv has been created


In [32]:
df = pd.read_csv('Data_CSV/ALL_EMOJI_LIST.csv',header=None)

df.head()


,0,1,2,3,4,5,6,7,8,9,...,3076,3077,3078,3079,3080,3081,3082,3083,3084,3085
0,🔹,✅,✅,✅,✅,🚀,🔹,✅,✅,✅,...,5️⃣,6️⃣,💡,🚀,❗,🚀,💼,✨,🔑,👉


### Getting the names for emojis

In [21]:
emoji_string = "".join(df.iloc[0])

unique_emojis = list(set(char for char in emoji_string if emoji.is_emoji(char)))

emoji_names = {e: emoji.demojize(e).strip(":").replace("_", " ").title() for e in set(emoji_string) if emoji.is_emoji(e)}

emoji_named_df = pd.DataFrame([unique_emojis], columns=emoji_names)

emoji_named_df


,✉,🎙,✔,🙈,🗣,🚀,🌐,👽,👾,🎁,...,🐫,🦗,☑,😍,😃,💥,💯,🕰,🤹,🤗
0,✉,🎙,✔,🙈,🗣,🚀,🌐,👽,👾,🎁,...,🐫,🦗,☑,😍,😃,💥,💯,🕰,🤹,🤗


In [31]:
sorted_emoji = {name: [] for name in emoji_names.values()}  
not_sorted_emoji = []

for char in emoji_string:
    if char in emoji_names:
        sorted_emoji[emoji_names[char]].append(char)
    else:
        not_sorted_emoji.append(char)

# Thanks CHATGPT!
max_length = max(len(val) for val in sorted_emoji.values())

for key in sorted_emoji:
    sorted_emoji[key] += [""] * (max_length - len(sorted_emoji[key]))

print(sorted_emoji)
print(not_sorted_emoji)

sorted_emoji_df = pd.DataFrame(sorted_emoji)

sorted_emoji_df.head()

{'Envelope': ['✉', '✉', '✉', '✉', '✉', '✉', '✉', '✉', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''

,Envelope,Studio Microphone,Check Mark,See-No-Evil Monkey,Speaking Head,Rocket,Globe With Meridians,Alien,Alien Monster,Wrapped Gift,...,Two-Hump Camel,Cricket,Check Box With Check,Smiling Face With Heart-Eyes,Grinning Face With Big Eyes,Collision,Hundred Points,Mantelpiece Clock,Person Juggling,Smiling Face With Open Hands
0,✉,🎙,✔,🙈,🗣,🚀,🌐,👽,👾,🎁,...,🐫,🦗,☑,😍,😃,💥,💯,🕰,🤹,🤗
1,✉,,✔,,🗣,🚀,🌐,👽,👾,🎁,...,,,☑,,,💥,💯,,,
2,✉,,✔,,🗣,🚀,🌐,,,🎁,...,,,☑,,,💥,💯,,,
3,✉,,✔,,🗣,🚀,🌐,,,,...,,,☑,,,💥,💯,,,
4,✉,,✔,,,🚀,🌐,,,,...,,,,,,💥,💯,,,
